In [1]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [11]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [19]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [23]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets2.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "FsLTwZOahu072Naat2M4gbX9h"
    consumer_secret = "QM11Xwc58xqSD8bnjOBiF8cv6oWZwYZmvfvgQMFxoyxolj32Wk"
    access_token = "1517590492761845760-1sQQPpLcOCDTe7II3GsUUhY6JingMD"
    access_token_secret = "e3rfHk90QafWbmIt0TgtX7CGWYHZDENhykAolxgN7Q8X9"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    # LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384475,  # Contiguous US
    #              -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
    #              -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii
    LOCATIONS = [-124.300047, 32.528759, -117.021459, 49.017836]  # West Coast

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(track=["Amber Heard"], locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517617407820374016, 'username': 'singleserving', 'created_at': 'Fri Apr 22 21:32:40 +0000 2022', 'lng': -118.4119065, 'lat': 34.0207895, 'text': '@CordyCord_ @Barbell_nft @HAPEsocial Your goodie bag was ganked! We had to put together what we could find from our… https://t.co/VfGsc7SOrr'}
{'id': 1517617407807811585, 'username': 'ClodzRamos', 'created_at': 'Fri Apr 22 21:32:40 +0000 2022', 'lng': -118.4119065, 'lat': 34.0207895, 'text': 'New chapter 🌴 https://t.co/3njoMSIWZh'}
{'id': 1517617409456148480, 'username': 'Fwiz', 'created_at': 'Fri Apr 22 21:32:41 +0000 2022', 'lng': -119.30660750000001, 'lat': 37.2691755, 'text': '@Tran @SpinnaSpence @Cole_Cook They’re out their now about to snap hook their approach shot into 16. 🤝Guest bedro… https://t.co/88ba2Fbhe1'}
{'id': 1517617411385544704, 'username': 'LauraTa36574495', 'created_at': 'Fri Apr 22 21:32:41 +0000 2022', 'lng': -119.7824645, 'lat': 36.786041999999995, 'text': '@MdBreathe @joegooding 😔😔 So sad'}
{'id': 1517617415193

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
